In [101]:
# ────────────────────────────────────────────────
# 1. Cargar librerías necesarias
# ────────────────────────────────────────────────

import os
# Permite acceder a variables de entorno (como claves de API en un archivo .env)

import time
# Permite hacer pausas controladas (ej. time.sleep(1) = espera 1 segundo) para no saturar APIS.

import pandas as pd  
# Permite trabajar con tablas (como Excel, pero en código)

import spotipy  
# Librería oficial para conectarse a Spotify

from spotipy.oauth2 import SpotifyClientCredentials  
# OAuth es un protocolo de autorización seguro que permite a tu app acceder a Spotify usando credenciales de cliente (ID + secreto)

from dotenv import load_dotenv  
# → Herramienta para leer el archivo '.env' donde guardas tus claves secretas
# Así no las dejas expuestas en el código


In [56]:
pip show python-dotenv

Name: python-dotenv
Version: 1.2.1
Summary: Read key-value pairs from a .env file and set them as environment variables
Home-page: 
Author: 
Author-email: Saurabh Kumar <me+github@saurabh-kumar.com>
License: 
Location: c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: 
Required-by: webdriver-manager
Note: you may need to restart the kernel to use updated packages.


In [57]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [117]:
# ════════════════════════════════════════════════════════════════════════════
# 2. CARGAR Y VALIDAR CREDENCIALES DE SPOTIFY
# ════════════════════════════════════════════════════════════════════════════

# Carga las variables de entorno desde el archivo .env
# 'override=True' → Reemplaza valores existentes, útil al actualizar credenciales
load_dotenv('.env', override=True)

# Extrae las credenciales del entorno
id_spotify = os.getenv('SPOTIFY_CLIENT_ID')        # ID público de tu aplicación
secreto_spotify = os.getenv('SPOTIFY_CLIENT_SECRET')  # Clave secreta (¡nunca la compartas!)

# Validación de seguridad: Verifica que ambas credenciales existan
if not id_spotify or not secreto_spotify:
    raise ValueError(
        "Error: Faltan credenciales en el archivo .env"
        "   Asegúrate de tener:"
        "   - SPOTIFY_CLIENT_ID=tu_client_id"
        "   - SPOTIFY_CLIENT_SECRET=tu_client_secret"
    )

# Crea el objeto de autenticación con las credenciales validadas
mis_credenciales = SpotifyClientCredentials(
    client_id=id_spotify,
    client_secret=secreto_spotify
)

# Inicializa la conexión con la API de Spotify
spotify = spotipy.Spotify(auth_manager=mis_credenciales)
print("Conexión exitosa a la API de Spotify")

# ────────────────────────────────────────────────
# En este punto ya podemos usar spotify para hacer consultas:
# - spotify.search() → Buscar canciones, artistas, álbumes
# - spotify.artist() → Información de un artista
# - spotify.track() → Detalles de una canción
# ────────────────────────────────────────────────


Conexión exitosa a la API de Spotify


In [118]:
# ────────────────────────────────────────────────
# 3. PRUEBA DE CONEXIÓN
# ────────────────────────────────────────────────
# Búsqueda de prueba: flamenco del 2016, 1 resultado
prueba = spotify.search(
    q='year:2016 genre:flamenco',  # Consulta: año 2016 y género flamenco
    type='track',                   # Tipo: canciones
    limit=1                         # Solo 1 resultado
)

# Si llega aquí, la conexión fue exitosa
print("✓ Conexión exitosa a la API de Spotify")


✓ Conexión exitosa a la API de Spotify
